In [1]:

import sqlite3
import pandas as pd
import json
import re

import glob
from tqdm import tqdm


In [2]:
exp_names = ['2jfm3ly9', 'tcgmxr0q', 'ksafxcri']
exp_params = []

for exp_name in exp_names:
    config_locs= f'/home/nys/nni-experiments/{exp_name}/trials/*/parameter.cfg'
    config_locs = glob.glob(config_locs)

    params = []
    for config_loc in config_locs:
        trialJobId = config_loc.split('/')[-2]
        cur_config = json.loads(open(config_loc).read())['parameters']
        cur_config.update({'trialJobId': trialJobId})
        params.append(cur_config)
    
    exp_params.append(pd.DataFrame(params))


In [3]:
metric_datas = []

for exp_name in exp_names:
    con = sqlite3.connect(f'/home/nys/nni-experiments/{exp_name}/db/nni.sqlite')
    # cur = con.cursor()
    # cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    # print(cur.fetchall())

    metric_data = pd.read_sql_query("SELECT * FROM MetricData;",con)
    metric_datas.append(metric_data)


In [4]:
experiments = []

In [5]:
for i in range(len(exp_names)):
    cur_df = pd.merge(exp_params[i], metric_datas[i], on='trialJobId', how= "right")
    experiments.append(cur_df)
    



In [6]:
for i in range(len(experiments)):
    experiments[i]['data'] = experiments[i]['data'].str.replace("\"", "").astype(float)
    experiments[i] = experiments[i].sort_values(by=['data', 'factors'], ascending=[False, True])

In [7]:

als_df = experiments[0]
bpr_df = experiments[1]
lmf_df = experiments[2]


display(als_df.head(10))
display(bpr_df.head(10))
display(lmf_df.head(10))


,use_week_num,use_full_item,model,factors,regularization,bm25_weighting,use_gpu,use_cg,iterations,trialJobId,timestamp,parameterId,type,sequence,data
98,5,False,als,186,0.0100,True,False,True,24,UOzB5,1657612782436,98,FINAL,0,0.008695
79,5,False,als,186,0.0001,True,False,True,24,s4mzA,1657611404205,79,FINAL,0,0.008692
48,5,False,als,186,0.0001,True,False,True,26,xxKAi,1657608843369,48,FINAL,0,0.008688
99,5,False,als,186,0.0010,True,False,True,24,g2koU,1657612860387,99,FINAL,0,0.008686
213,5,False,als,186,0.0100,True,False,False,24,SXkuL,1657620189573,213,FINAL,0,0.008667
80,5,False,als,186,0.0001,True,False,False,24,cYsHK,1657611618047,80,FINAL,0,0.008665
61,5,False,als,186,0.0001,True,False,False,26,R6Aqr,1657609899213,61,FINAL,0,0.008665
149,5,False,als,186,0.1000,True,False,False,26,V9DfN,1657616321187,149,FINAL,0,0.008663
44,5,False,als,186,0.1000,True,False,True,26,xERgU,1657608623657,44,FINAL,0,0.008659
243,5,False,als,186,0.0100,True,False,True,19,cYBBh,1657621910039,243,FINAL,0,0.008650


,use_week_num,use_full_item,use_gpu,model,learning_rate,iterations,regularization,factors,verify_negative_samples,trialJobId,timestamp,parameterId,type,sequence,data
232,4,False,True,bpr,0.99,46,0.0100,10,False,Sa75E,1657706389225,249,FINAL,0,0.018244
197,4,False,True,bpr,0.99,46,0.0001,11,False,yxPai,1657704718780,210,FINAL,0,0.018244
198,4,False,True,bpr,0.99,92,0.0001,11,False,J384n,1657704759211,211,FINAL,0,0.018244
261,4,False,True,bpr,0.99,120,0.0001,11,False,ZOOmB,1657707760190,280,FINAL,0,0.018244
283,4,False,True,bpr,0.99,122,0.0001,11,False,HUfHa,1657708764824,303,FINAL,0,0.018244
202,4,False,True,bpr,0.99,92,0.0001,23,False,LKfZe,1657704984248,216,FINAL,0,0.018244
217,4,False,True,bpr,0.99,92,0.0001,23,True,tDHbx,1657705679698,232,FINAL,0,0.018244
244,4,False,True,bpr,0.99,75,0.0001,23,False,ttrJj,1657707014859,262,FINAL,0,0.018244
251,4,False,True,bpr,0.99,91,0.0001,23,True,t7X5K,1657707374910,270,FINAL,0,0.018244
262,4,False,True,bpr,0.99,92,0.0010,23,False,lx1nM,1657707795232,281,FINAL,0,0.018244


,use_week_num,use_full_item,use_gpu,model,learning_rate,iterations,regularization,factors,neg_prop,trialJobId,timestamp,parameterId,type,sequence,data
61,1,False,False,lmf,0.99,85,0.1,117,24,bghX8,1657642006380,61,FINAL,0,0.008299
98,1,False,False,lmf,0.99,85,0.1,150,84,J1dYK,1657645174216,98,FINAL,0,0.008249
111,1,False,False,lmf,0.99,63,0.1,117,24,gDZgJ,1657646177372,111,FINAL,0,0.008125
180,1,False,False,lmf,0.99,85,0.1,150,65,GDDzC,1657651914686,180,FINAL,0,0.008113
51,1,False,False,lmf,0.99,85,0.1,117,84,t7yEr,1657640991086,51,FINAL,0,0.008108
47,1,False,False,lmf,1.50,129,0.1,117,84,CMYBc,1657640645472,47,FINAL,0,0.008099
88,1,False,False,lmf,0.99,57,0.1,117,84,R8fui,1657644280537,88,FINAL,0,0.008087
119,1,False,False,lmf,0.99,116,0.1,117,84,uYqkW,1657646918252,119,FINAL,0,0.008084
158,1,False,False,lmf,0.99,85,0.1,117,54,hjgsF,1657649791188,158,FINAL,0,0.008079
44,1,False,False,lmf,0.99,133,0.1,117,84,h4mrz,1657640316273,44,FINAL,0,0.008074


In [8]:
als_df.to_csv('../../data/implicit/als_df.csv', index = False)
bpr_df.to_csv('../../data/implicit/bpr_df.csv', index = False)
lmf_df.to_csv('../../data/implicit/lmf_df.csv', index = False)

In [9]:
metric_cols = list(metric_datas[0].columns)
exp_param_cols = [list(set(list(exp_param.columns)) - set(['trialJobId',
                                                           'use_week_num',
                                                           'use_full_item',
                                                           'bm25_weighting',
                                                           'model'])) for exp_param in exp_params]

# exp_param_cols = dict(zip(['als','bpr','lmf'], exp_param_cols))
print(metric_cols)
display(exp_param_cols)

['timestamp', 'trialJobId', 'parameterId', 'type', 'sequence', 'data']


[['iterations', 'factors', 'use_gpu', 'regularization', 'use_cg'],
 ['iterations',
  'factors',
  'use_gpu',
  'verify_negative_samples',
  'regularization',
  'learning_rate'],
 ['iterations',
  'factors',
  'use_gpu',
  'neg_prop',
  'regularization',
  'learning_rate']]

In [10]:
import argparse
import implicit
from tqdm import tqdm

from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k
from implicit.nearest_neighbours import bm25_weight
import os
import datetime

import numpy as np
import pandas as pd

import sys
sys.path.insert(0, '../../')
from src.utils.preprocess import *

In [11]:
seed = 2022
folder = '../../data/'
art_df = pd.read_csv(os.path.join(folder, "articles.csv"), dtype={'article_id': str})
cus_df = pd.read_csv(os.path.join(folder, "customers.csv"))
trans_df = pd.read_csv(os.path.join(folder, "transactions_train.csv"), dtype={'article_id': str})
trans_df = make_weeknum_col(trans_df)
trans_df['t_dat'] = pd.to_datetime(trans_df['t_dat'])
users = cus_df.customer_id.unique().tolist()
num2users = dict(list(enumerate(users)))
users2num = dict(zip(num2users.values(), num2users.keys()))

In [12]:
def to_user_item_coo(df, unique_users, unique_items):
    row = df['user_id'].values
    col = df['item_id'].values    
    data = np.ones(df.shape[0])
    coo = coo_matrix((data, (row, col)), shape=(len(unique_users), len(unique_items)))
    return coo

def preprocess_test(trans_df, art_df, users2num, **kwargs):
    use_week_num = kwargs['use_week_num']
    use_full_item = kwargs['use_full_item']
    start_date = trans_df.t_dat.max() - datetime.timedelta(7 * use_week_num)
    cur_trans_df = trans_df[trans_df['t_dat'] > start_date]

    if use_full_item == False:
        items = cur_trans_df.article_id.unique().tolist()
    else:
        items = art_df.article_id.unique().tolist()
        
    num2items = dict(list(enumerate(items)))
    items2num = dict(zip(num2items.values(), num2items.keys()))

    cur_trans_df['user_id'] = cur_trans_df['customer_id'].map(users2num)
    cur_trans_df['item_id'] = cur_trans_df['article_id'].map(items2num)

    coo_train = to_user_item_coo(cur_trans_df, users2num.values(), items)
    csr_train = coo_train.tocsr()

    return csr_train, num2items

def train(model_name, csr_train, bm25_weighting=None, random_state=2022, **kwargs):
    if model_name == 'als':
        if bm25_weighting:
            csr_train = (bm25_weight(csr_train, B=0.9) * 5).tocsr()
        cur_model = implicit.als.AlternatingLeastSquares(random_state=random_state,**kwargs)
    elif model_name == 'bpr':
        cur_model = implicit.bpr.BayesianPersonalizedRanking(random_state=random_state,**kwargs)
    elif model_name == 'lmf':                    
        cur_model = implicit.lmf.LogisticMatrixFactorization(random_state=random_state,**kwargs)
    
    cur_model.fit(csr_train, show_progress=False)
    return cur_model

In [13]:
best_configs = [df.iloc[0].to_dict() for df in experiments]

In [14]:
import warnings
warnings.filterwarnings('ignore')

model_info_data = []

for i, best_config in enumerate(best_configs):

    model_name = best_config['model']
    model_kwargs = {k:best_config[k] for k in exp_param_cols[i]}
    csr_train, num2items = preprocess_test(trans_df, art_df, users2num, **best_config)
    print(model_name, best_config)    
    print(csr_train.shape)
    bm25_weighting = None if "bm25_weighting" not in best_config.keys() else best_config["bm25_weighting"]
    print(bm25_weighting)
    trained_model = train(model_name, csr_train, bm25_weighting, **model_kwargs)
    model_info_data.append((trained_model, num2items, csr_train))
    print(model_name, "finished")
    print("="*80)

als {'use_week_num': 5, 'use_full_item': False, 'model': 'als', 'factors': 186, 'regularization': 0.01, 'bm25_weighting': True, 'use_gpu': False, 'use_cg': True, 'iterations': 24, 'trialJobId': 'UOzB5', 'timestamp': 1657612782436, 'parameterId': '98', 'type': 'FINAL', 'sequence': 0, 'data': 0.008695135026110617}
(1371980, 30482)
True
als finished
bpr {'use_week_num': 4, 'use_full_item': False, 'use_gpu': True, 'model': 'bpr', 'learning_rate': 0.99, 'iterations': 46, 'regularization': 0.01, 'factors': 10, 'verify_negative_samples': False, 'trialJobId': 'Sa75E', 'timestamp': 1657706389225, 'parameterId': '249', 'type': 'FINAL', 'sequence': 0, 'data': 0.018244007836486192}
(1371980, 28297)
None
bpr finished
lmf {'use_week_num': 1, 'use_full_item': False, 'use_gpu': False, 'model': 'lmf', 'learning_rate': 0.99, 'iterations': 85, 'regularization': 0.1, 'factors': 117, 'neg_prop': 24, 'trialJobId': 'bghX8', 'timestamp': 1657642006380, 'parameterId': '61', 'type': 'FINAL', 'sequence': 0, 'dat

In [ ]:
a = 

In [15]:
def make_prediction(model, csr_train, num2items , num2users):
    preds = []
    batch_size = 2000
    to_generate = np.arange(len(num2users.keys()))
    for startidx in tqdm(range(0, len(to_generate), batch_size)):
        batch = to_generate[startidx : startidx + batch_size]
        # filter_already_liked_items를 테스트 할 때는 끈다!
        # 검증과 다름?
        ids, scores = model.recommend(batch, csr_train[batch], N=12, filter_already_liked_items=False)
        for i, userid in enumerate(batch):
            customer_id = num2users[userid]
            user_items = ids[i]
            article_ids = [str(num2items[item_id]) for item_id in user_items]
            preds.append((customer_id, ' '.join(article_ids)))

    df_preds = pd.DataFrame(preds, columns=['customer_id', 'prediction'])
    print(df_preds.shape)
    
    return df_preds

In [25]:
to_generate = np.arange(len(num2users.keys()))

In [27]:
ids, scores = cur_model.recommend(to_generate, cur_csr[to_generate], N=12, filter_already_liked_items=False)

In [47]:
temp = pd.DataFrame(ids)

In [48]:
temp.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,5770,3357,2266,28849,2416,10090,557,2381,1518,7900,2016,3901
1,11,10,9,8,7,6,5,4,3,2,1,0
2,3066,3685,1266,7115,3115,26758,5339,11506,12028,731,2928,12059
3,11,10,9,8,7,6,5,4,3,2,1,0
4,11,10,9,8,7,6,5,4,3,2,1,0


In [50]:
num2items[28849]

KeyError: 28849

In [32]:
prediction_df = pd.DataFrame(ids)
prediction_df = prediction_df.reset_index()

In [43]:
prediction_df.columns

Index(['index', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='object')

In [44]:
for i in range(12):

    prediction_df[i] = prediction_df[i].map(num2items)

        

In [46]:
prediction_df.head()

,index,0,1,2,3,4,5,6,7,8,9,10,11
0,0,0807241062,0684238003,0802153005,NaN,0870289003,0846139002,0862693001,0827968002,0822171001,0821152001,0694298007,0767032001
1,1,0863937003,0579541001,0869379002,0890631002,0837443001,0759814034,0610776105,0562245001,0237347060,0889669006,0913272003,0786022008
2,2,0725112001,0871311003,0928152003,0714807003,0568601023,NaN,0897054002,0859363001,0744036003,0826492003,0880481002,0873000004
3,3,0863937003,0579541001,0869379002,0890631002,0837443001,0759814034,0610776105,0562245001,0237347060,0889669006,0913272003,0786022008
4,4,0863937003,0579541001,0869379002,0890631002,0837443001,0759814034,0610776105,0562245001,0237347060,0889669006,0913272003,0786022008


In [30]:
ids[0]

array([ 5770,  3357,  2266, 28849,  2416, 10090,   557,  2381,  1518,
        7900,  2016,  3901], dtype=int32)

In [20]:
cur_model, cur_num2items, cur_csr = model_info_data[0]
predicted_df = make_prediction(cur_model, cur_csr, cur_num2items, num2users)

 51%|█████████████████████████▏                       | 353/686 [00:49<00:46,  7.13it/s]


KeyboardInterrupt: 

In [ ]:
cur_model, cur_num2items, cur_csr = model_info_data[2]
predicted_df2 = make_prediction(cur_model, cur_csr, cur_num2items, num2users)

In [ ]:
cur_model, cur_num2items, cur_csr = model_info_data[1]
predicted_df1 = make_prediction(cur_model, cur_csr, cur_num2items, num2users)

In [18]:
predicted_dfs = []
for cur_model, cur_num2items, cur_csr in model_info_data:
    predicted_df = make_prediction(cur_model, cur_csr, cur_num2items, num2users)
    predicted_dfs.append(predicted_df)

100%|█████████████████████████████████████████████████| 686/686 [01:36<00:00,  7.12it/s]


(1371980, 2)


  0%|                                                           | 0/686 [00:00<?, ?it/s]


KeyError: -1

In [19]:
cur_num2items

{0: '0905507002',
 1: '0772902011',
 2: '0867407002',
 3: '0911214001',
 4: '0572998001',
 5: '0832253002',
 6: '0885951003',
 7: '0809846001',
 8: '0787946002',
 9: '0736870001',
 10: '0874816003',
 11: '0610776002',
 12: '0751471001',
 13: '0452818001',
 14: '0768912001',
 15: '0874754016',
 16: '0860797003',
 17: '0751471039',
 18: '0882900007',
 19: '0746330004',
 20: '0860720001',
 21: '0904960001',
 22: '0879248001',
 23: '0854963005',
 24: '0710278004',
 25: '0842785001',
 26: '0849114001',
 27: '0883810001',
 28: '0862958001',
 29: '0640542002',
 30: '0835348011',
 31: '0902161005',
 32: '0706016001',
 33: '0914441005',
 34: '0921266005',
 35: '0921266007',
 36: '0876733001',
 37: '0921266006',
 38: '0921298003',
 39: '0881751002',
 40: '0682236001',
 41: '0845790002',
 42: '0915529001',
 43: '0767862001',
 44: '0816563004',
 45: '0822344001',
 46: '0827687001',
 47: '0852584001',
 48: '0760084003',
 49: '0864040001',
 50: '0776887001',
 51: '0871376002',
 52: '0753737001',
 53